In [ ]:
library(DESeq2)

# Methods section 2.5

In [ ]:
# DESeq analysis for mRNA+RPF of DS3
# split by the 3 metrics used; 60 first nt, full-length CDS, full-length CDS minus TY1-GAG
for(metric in c('60nt','mrna','Ty1')){
    # import count table derived from Plastid
    count.table <- read.table(paste('~/mhenault_landrylab/md_ty_expression/deseq2/', metric, '.counts.tab', sep=''), header=TRUE, row.names='region_name')
    #import design table
    design.table <- read.table('~/mhenault_landrylab/md_ty_expression/deseq2/samples.tab', header=TRUE, row.names='X0', sep='\t')
    design.table$Experiment <- factor(design.table$Experiment, levels=c('rna','rpf'))
    design.table$Species <- factor(design.table$Species, ordered=FALSE)
    
    #split the analysis per subgenome (subsampling both samples and genes accordingly)
    for (subgenome in c('S288c','CBS432')){
        if (subgenome == 'S288c'){
            sub.design.table <- design.table[design.table$Species %in% c('Scer','hyb'), ]
            sub.design.table$Species <- relevel(sub.design.table$Species, ref='Scer')
            sub.count.table <- count.table[grep(subgenome, rownames(count.table)), rownames(sub.design.table)]
        }
        if (subgenome == 'CBS432'){
            sub.design.table <- design.table[design.table$Species %in% c('Spar','hyb'), ]
            sub.design.table$Species <- relevel(sub.design.table$Species, ref='Spar')
            sub.count.table <- count.table[grep(subgenome, rownames(count.table)), rownames(sub.design.table)]
        }

        dat <- DESeqDataSetFromMatrix(countData=sub.count.table, colData=sub.design.table, design= ~ Experiment + Species + Experiment:Species)
        res <- DESeq(dat)

        #parse relevant contrasts and export results (raw and shrinked LFC)
        for(f in resultsNames(res)[2:4]){
            Res <- results(res, name=f)
            write.csv(Res, paste('~/mhenault_landrylab/md_ty_expression/deseq2/results.',subgenome,'.',metric,'.raw_lfc.', f, '.csv', sep=''))

            ResLFC <- lfcShrink(res, coef=f, type='apeglm')
            write.csv(ResLFC, paste('~/mhenault_landrylab/md_ty_expression/deseq2/results.',subgenome,'.',metric,'.shrink_lfc.', f, '.csv', sep=''))
        }
        
        # export normalized counts
        if(metric == 'mrna'){
            write.csv(counts(res, normalize=TRUE), paste('~/mhenault_landrylab/md_ty_expression/deseq2/norm_counts.',subgenome,'.',metric,'.csv', sep=''))
        }
    }
}

In [ ]:
# DESeq analysis for mRNA+RPF of DS3
# select only samples for mRNA-seq, excluding RPF data
samples.rnaseq <- rownames(subset(design.table, Experiment=='rna'))
# select only one metric, full-length CDS
metric <- 'mrna'
# import count table derived from Plastid
count.table <- read.table(paste('~/mhenault_landrylab/md_ty_expression/deseq2/', metric, '.counts.tab', sep=''), header=TRUE, row.names='region_name')
#import design table
design.table <- read.table('~/mhenault_landrylab/md_ty_expression/deseq2/samples.tab', header=TRUE, row.names='X0', sep='\t')
design.table$Species <- factor(design.table$Species, ordered=FALSE)
#split the analysis per subgenome (subsampling both samples and genes accordingly)
for (subgenome in c('S288c','CBS432')){
    if (subgenome == 'S288c'){
        sub.design.table <- design.table[design.table$Species %in% c('Scer','hyb') & design.table$Experiment=='rna', ]
        sub.design.table$Species <- relevel(sub.design.table$Species, ref='Scer')
        sub.count.table <- count.table[grep(subgenome, rownames(count.table)), rownames(sub.design.table)]
    }
    if (subgenome == 'CBS432'){
        sub.design.table <- design.table[design.table$Species %in% c('Spar','hyb') & design.table$Experiment=='rna', ]
        sub.design.table$Species <- relevel(sub.design.table$Species, ref='Spar')
        sub.count.table <- count.table[grep(subgenome, rownames(count.table)), rownames(sub.design.table)]
    }
    
    dat <- DESeqDataSetFromMatrix(countData=sub.count.table, colData=sub.design.table, design= ~ Species)
    res <- DESeq(dat)

    #parse relevant contrast and export results (raw and shrinked LFC)
    f <- resultsNames(res)[2]
    Res <- results(res, name=f)
    write.csv(Res, paste('~/mhenault_landrylab/md_ty_expression/deseq2/results.',subgenome,'.raw_lfc.', f, '.rnaseq.csv', sep=''))

    ResLFC <- lfcShrink(res, coef=f, type='apeglm')
    write.csv(ResLFC, paste('~/mhenault_landrylab/md_ty_expression/deseq2/results.',subgenome,'.shrink_lfc.', f, '.rnaseq.csv', sep=''))

    # export normalized counts
    write.csv(counts(res, normalize=TRUE), paste('~/mhenault_landrylab/md_ty_expression/deseq2/norm_counts.',subgenome,'.rnaseq.csv', sep=''))
    
}